# Determining the MSCI-ESG Rating 

## Semantic similarity computation

In [2]:
import semantic_eval
import pandas as pd

# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython

%load_ext autoreload
# notebook will reload external python modules;
%autoreload 2 
# make figures appear inline
# %matplotlib inline 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Baseline with LSI

using paragraph tokenizer

In [3]:
res_lsi = semantic_eval.compute_baseline("./ccpv230130.pdf")
res_df_with_lsi = semantic_eval.get_results_dataframe_lsi(res_lsi)
res_df_with_lsi.sort_values(by="score", ascending=False)

,index,text,key_term,score
79,79,préavis n° 2023/91 \ncampagne de comptages 202...,Changement climatique [Pillier Environnemental],0.323661
8,8,mme la municipale stéphanie schmutz répon...,Opportunités sociales [Pillier Sociale],0.270042
46,46,"au vote nominal, le conseil décide par ...",Opportunités sociales [Pillier Sociale],0.234605
62,62,il pourrait poursuivre en évoquant le ratage e...,Opportunités sociales [Pillier Sociale],0.230354
71,71,m. alexander federau intervient au sujet de la...,Opportunités sociales [Pillier Sociale],0.229585
...,...,...,...,...
91,92,résultats individuels : \nalberti marina \nbes...,Comportement d'entreprise [Pillier de la gouve...,0.000105
92,93,oui \noui \noui \noui \noui \n- \noui \noui \n...,Comportement d'entreprise [Pillier de la gouve...,0.000102
90,91,non \n- \nnon \noui \noui \noui \noui \n- \nou...,pollution et déchets [Pillier Environnemental],0.000037
93,95,oui \n- \noui \nabstention \noui \noui \noui \...,Gouvernance d'entreprise [Pillier de la gouver...,0.000036


In [4]:
f = open('./results/lsi_results.txt', 'w', encoding='utf-8')
for i in range(len(res_df_with_lsi)):
    f.write(res_df_with_lsi['key_term'][i] + "\n")
    f.write(str(res_df_with_lsi['score'][i]) + " : " + res_df_with_lsi['text'][i] + "\n----\n")
f.close()


### Classification with paragraphs

In [6]:
_,df_results_with_paragraphs = semantic_eval.main()
# res_df_with_paragraphs = semantic_eval.get_results_dataframe(results_with_paragraphs, sort='avg_score')
# res_df_with_paragraphs.to_csv('./results/semantic_eval_with_paragraphs.csv', index=False)
# res_df_with_paragraphs
# individual_sentence_scores_df = pd.DataFrame(individual_sentence_scores)
df_results_with_paragraphs.sort_values(by="score", ascending=False)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\azeem\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\azeem\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Preprocessing...
Loading model...
Embedding...
Calculating similarity...
Classifying paragraphs...


,paragraph,score,key_term
32,à un projet intéressant. des projets similair...,0.438676,Opportunités Environnementals [Pillier Environ...
41,m. jacky colomb pense que la propositio...,0.409219,Opportunités Environnementals [Pillier Environ...
89,mme la municipale roxane faraut lui répond qu’...,0.399220,Opportunités Environnementals [Pillier Environ...
29,"il n’y a pas de \nraison ni d’enlaidir, ni d’a...",0.398473,Opportunités Environnementals [Pillier Environ...
112,préavis n° 2023/94 \nprojet d’aménagements en ...,0.384884,Opportunités Environnementals [Pillier Environ...
...,...,...,...
11,mme la municipale répond également à m. ...,0.163502,capital humain [Pillier Sociale]
92,m. le syndic daniel rossellat lui répond qu’ef...,0.145744,Changement climatique [Pillier Environnemental]
0,"m. le syndic daniel rossellat, mmes les munici...",0.143312,pollution et déchets [Pillier Environnemental]
100,"mme caroline bordier se rappelle, au début de ...",0.126672,Capital Naturel [Pillier Environnemental]


In [7]:
# print all sentences like the following score:sentence and new line and save to text file
f = open('./results/pre-trained_model_results.txt', 'w',encoding='utf-8')
for i in range(len(df_results_with_paragraphs)):
    f.write(df_results_with_paragraphs['key_term'][i] + "\n")
    f.write(str(df_results_with_paragraphs['score'][i]) + " : " + df_results_with_paragraphs['paragraph'][i] + "\n----\n")
f.close()

## Sentiment Analysis

In [60]:
from transformers import pipeline
from tqdm import tqdm

analyzer = pipeline(
    task='text-classification',
    model="cmarkea/distilcamembert-base-sentiment",
    tokenizer="cmarkea/distilcamembert-base-sentiment"
)

MSCI_rating = {
    'AAA': [8.571,10],
    'AA': [7.143,8.571],
    'A': [5.714,7.143],
    'BBB': [4.286,5.714],
    'BB': [2.857,4.286],
    'B': [1.429,2.857],
    'CCC': [0,1.429]
}

# def compute_rating(sentence,analyzer, return_all_scores=False):
#     result = analyzer(sentence, return_all_scores=return_all_scores)
#     if return_all_scores:
#         rating = sum(int(r['label'].split()[0]) * r['score'] for r in result[0])*2
#     else:
#         rating = int(result[0]['label'].split()[0])*2
    
#     # get the number rating from the pipeline result
    
#     return rating

def compute_rating(text, analyzer):
    result = analyzer(text, top_k=1)
    score = int(result[0]['label'].split()[0])
    return score


def get_msci_rating(expected_rating):
    for rating, bounds in MSCI_rating.items():
        if bounds[0] <= expected_rating <= bounds[1]:
            return rating

res_df = df_results_with_paragraphs.copy()
res_df['rating'] = 0
res_df['MSCI_rating'] = None

for i,kt in tqdm(enumerate(res_df['key_term'])):
    rating = compute_rating(res_df['paragraph'][i], analyzer)*2
    # for text in res_df['paragraph'][i]:
    #     rating += compute_rating(text, analyzer)
    # avg_rating = rating/len(res_df['paragraph'][i])
    res_df.loc[i, 'rating'] = round(rating, 2)
    res_df.loc[i, 'MSCI_rating'] = get_msci_rating(res_df.loc[i, 'rating'])

118it [00:11, 10.59it/s]


The following result is obtained by simply taking highest note from the output that is out of 5 and multipying it by 2 to get a rating out of 10. We then compute the average for each category. 

In [61]:
print(f"\t\t\t MSCI-ESG Rating for CCPV-230130")
res_df

			 MSCI-ESG Rating for CCPV-230130


,paragraph,score,key_term,rating,MSCI_rating
0,"m. le syndic daniel rossellat, mmes les munici...",0.143312,pollution et déchets [Pillier Environnemental],6,A
1,"adnane alexandre, braunwalder alex, bürki ...",0.091231,Changement climatique [Pillier Environnemental],10,AAA
2,aucune demande de modification n’étant de...,0.202984,Opposition des parties prenantes [Pillier Soci...,8,AA
3, m. le président annonce formellement qu’il ...,0.303946,Capital Naturel [Pillier Environnemental],2,B
4, m. le président donne lecture de la lettre ...,0.176049,pollution et déchets [Pillier Environnemental],8,AA
...,...,...,...,...,...
113,"mme la municipale stéphanie schmutz, en intro...",0.205843,capital humain [Pillier Sociale],6,A
114,"entre autres, la \nligne nyon-eysins comporte ...",0.207711,Changement climatique [Pillier Environnemental],6,A
115,la ville de nyon est en contact étroit avec la...,0.216202,capital humain [Pillier Sociale],8,AA
116,"m. yves gauthier-jaques, à la suite de ...",0.265000,Opposition des parties prenantes [Pillier Soci...,6,A


In [65]:
f = open('./results/msci_rating.txt', 'w', encoding='utf-8')
for i in range(len(res_df)):
    f.write(f"{ res_df['key_term'][i]} {str(round(df_results_with_paragraphs['score'][i],2))} : | {res_df['rating'][i]}/10 ({res_df['MSCI_rating'][i]})" + "\n")
    f.write(  res_df['paragraph'][i] + "\n----\n")
f.close()

the result of the output from the model gives us the the probability for each note out of 5. So in this case, we multiply each mark by its probability and then multiply it by 2. And then we compute the average marks for each category.

In [ ]:
print(f"\t\t\t MSCI-ESG Expected Rating for CCPV-230130. Sorted by most popular topics (avg_score)")
res_df